함수형 프로그래밍은 상태가 없는 프로그래밍을 강조하낟. 파이썬에서 이는 제너레이터 식, 제너레이터 함수, 반복 가능 객체를 사용한 프로그래밍으로 귀결된다. 이번 장에서는 반복 가능한 컬렉션에 대한 작업을 도화주는 다양한 함수를 제공하는 itertools 라이브러리에 대한 공부를 계속할 것이다. 

이번 젱어세는 순열과 조합을 다루는 itertools 함수를 살펴본다. 그러한 함수를 사용하는 몇 가지 요리법과 함수들을 살펴본다. 다음과 같은 함수가 이러한 범주에 들어간다. 

* product(): 이 함수는 내포된 for 루프를 사용해 만들어 내는 것과 같은 데카르트 곱을 생성한다.

* permutations(): 이 함수는 p개의 원소가 있는 원집합으로부터 순서를 감안해 만들어 낼 수 있는 길이가 r인 튜플을 모두 생성한다.

* combnations(): 이 함수는 p개의 원소가 있는 원집합으로부터, 정렬한 순서로 길이가 r인 튜플을 모두 생성한다. 이때 중복된 원소가 없어야 한다.

* combinations_with_replacement(): 이 함수는 p개의 원소가 있는 원집합으로부터, 정렬한 순서로 길이가 r인 튜플을 모두 생성한다. 이때 원소의 중복을 허용한다.

이러한 함수들은 작은 입력 데이터로부터 매우 큰 결과 집합을 만들어 낼 사능성이 있는 알고리즘을 구현한다. 문제의 종류에 따라서는 엄청나게 커질 수 있는 순열의 집항을 일일히 열거하면서 정확한 해법을 찾을 수 있는 경우도 있다. 여기세 있는 함수들은 큰 순열을 만드는 작업을 매우 쉽게 해준다. 하지만 그 단순성으로 인해 실제로는 최적이 아닌 경우도 있다.

### 데카르트 곱 열거하기

데카르트 곱이라는 용어는 몇 개의 집합에서 가능한 모든 원소의 좁합을 열거한다는 아이디어에서 비롯된 것이다.

수학적으로, 두 집합의 곱인 {1, 2, 3, ..., 13} * {C, D, H, S}에는 52개의 원소쌍이 들어간다.

앞의 결과를 다음과 같은 명령으로 만들어 낼 수 있다.

In [1]:
from itertools import product

list(product(range(1, 14), "CDHS"))

[(1, 'C'),
 (1, 'D'),
 (1, 'H'),
 (1, 'S'),
 (2, 'C'),
 (2, 'D'),
 (2, 'H'),
 (2, 'S'),
 (3, 'C'),
 (3, 'D'),
 (3, 'H'),
 (3, 'S'),
 (4, 'C'),
 (4, 'D'),
 (4, 'H'),
 (4, 'S'),
 (5, 'C'),
 (5, 'D'),
 (5, 'H'),
 (5, 'S'),
 (6, 'C'),
 (6, 'D'),
 (6, 'H'),
 (6, 'S'),
 (7, 'C'),
 (7, 'D'),
 (7, 'H'),
 (7, 'S'),
 (8, 'C'),
 (8, 'D'),
 (8, 'H'),
 (8, 'S'),
 (9, 'C'),
 (9, 'D'),
 (9, 'H'),
 (9, 'S'),
 (10, 'C'),
 (10, 'D'),
 (10, 'H'),
 (10, 'S'),
 (11, 'C'),
 (11, 'D'),
 (11, 'H'),
 (11, 'S'),
 (12, 'C'),
 (12, 'D'),
 (12, 'H'),
 (12, 'S'),
 (13, 'C'),
 (13, 'D'),
 (13, 'H'),
 (13, 'S')]

이러한 곱의 계산을 임의의 개수의 반복 가능 컬렉션으로 확장할 수 있다. 컬렉션의 수가 많아지면 결과 집합도 커진다.

### 곱을 축약하기

관계형 데이터베이스 이론에서 두 테이블의 조인 연산은 곱에 대한 필터링으로 생각할 수 있다. SQL에서 WHERE절이 없는 SELECT문은 여러 테이블의 각 행의 데카르트 곱을 만들어 낸다. 이 상황을 가장 최악의 경우, 즉 필요한 결과를 걸러내기 위한 아무런 필터가 없이 테이블 시아의 곱을 구하는 경우에 대한 알고리즘이라 생각할 수 있다.

join() 함수를 사용해 두 테이블을 다음과 같이 결합할 수 있다.

In [4]:
def join(t1, t2, where):
    return filter(where, product(t1, t2))

두 반복 가능 객체 t1과 t2의 모든 조합을 계산한다. filter() 함수는 조합한 결과에 where 함수를 적용하여 적당한 행을 선택할 것이다. where 함수가 단순한 불린값을 반환한다면 이 방식이 잘 작동한다.

단순히 열의 참 거짓에 따라 일치시키는 것과는 다른 일을 해야 할 경우도 있다. 그 대신 원소 사이의 최댓값과 최솟값을 검색하는 경우를 생각해보자.

PIL.Image 객체가 있다면, 픽셀의 컬렉션에 대해 다음과 같은 명령을 사용해 루프를 돌 수 있다.

In [5]:
def pixel_iter(img):
    w, h = img.size
    return ((c, img.getpixel(c)) for c in product(range(w), range(h)))

이미지 크기를 사용해 각 좌표의 범위를 결정했다. product(range(w), range(b))를 계산하는 것은 모든 가능한 좌표의 조합을 만들어 낸다. 이는 결과적으로 for 루프를 2개 중첩시킨 것과 같다.

이렇게 하면 각 픽셀과 좌표를 제공할 수 있다는 장점이 있다. 그 후 이 픽셀 정보를 특별한 순서로 보존하지 않고 처리하더라도 원이미지를 재구성할 수 있다. 부하를 여러 코어나 프로세서에 분산시키기 위해 다중프로세스나 다중 스레드를 사용하는 경우에는 이러한 방식이 좀 더 유용하다. concurrnet.futures 모듈은 부하를 여러 코어나 프로세서에 쉽게 분산할 수 있도록 도와준다.

### 거리 계산하기

의사 결정을 위해 가장 가까운 답을 찾아야 하는 경우가 많다. 이러한 경우에는 단순한 동등성 검사를 사용할 수는 없다. 그 대신 거리 지표를 사용하여 목적과 거리가 가장 가까운 원소를 찾아야 한다. 텍스트의 경우 레벤스타인 거리를 사용할 수 있다. 이 거리는 어떤 텍스트를 대상 텍스르로 바꾸기 위해 얼마나 많은 변경이 필요한지를 측정한다.

여기서는 좀 더 단순한 예제를 사용할 것이다. 이 계산에는 매우 단순한 수학이 들어가지만 그 단순성에도 불구하고, 주의깊게 구현하지 않으면 제대로 동작하지 않는다.

다음은 유클리드와 맨해튼 거리 함수다.

In [6]:
def euclidean(pixel, color):
    return math.sqrt(sum(map(lambda x, y:(x-y)**2, pixel, color.rgb)))

In [7]:
def manhattan(pixel, color):
    return sum(map(lambda x, y: abs(x-y), pixel, color.rgb))

유클리드 거리는 RGB 공간에서 세 점 사이의 직작삼각형의 빗변의 길이를 측정한다. 맨해튼 거리는 세 점 사이의 직작 삼각형의 각 변의 합을 구한다. 유클리드 거리는 더 정확한 값을 제공하며, 맨해튼 거리는 더 빠른 계산 속도를 제공한다.

앞으로 할 일을 생각해본다면, 우리의 목표는 다음과 같은 구조를 만드는 것이다. 개별 픽셀에 대해 각 픽셀의 색과 정해진 색 집합의 가용 색 사이의 거리를 계산할 수 있다.

### 모든 픽셀과 모든 색 얻기

어떻게 하면 모든 픽셀과 모든 색을 포함하는 구조를 얻을 수 있을까? 그에 대한 답은 간단하지만 최적은 아니다.

3,648*2,736 크기의 133크레욜라 색이 있는 그림의 경우, 반복 가능 객체에서 평가해야 할 원소의 수는 1,327,463,424개다. 이 distances 식이 만들어 내는 좁합은 십억 개 이상이다. 이 수가 그렇게 비실용적이지는 않다. 파이썬은 이 정도의 크기 데이터를 처리할 수 있다. 하지만 이 예제는 product(0 함수를 안이하게 사용하는 경우에 생길 수 있는 중요한 단점을 보여준다. 

전체 크기를 대략적으로 분석하지 않고 이러한 종류의 대구모 분석을 무작정 수행할 수는 없다. 다음은 이러한 계산을 1,000,000번 수행하는 것은 timeit으로 측졍한 결과다.

* 유클리드 거리 2.8
* 맨해튼 거리 1.8

1백만을 1십억 크기로키운다면 맨해튼 거리의 경우 1800초 즉 반 시간이 걸린다는 뜻이다. 유클리드 거리의 경우에는 46분이다. 이러한 종류의 처리에 사용하기에는 파이썬의 기본 수학 연산이 너무 느린 것 같다.

더 중요한 점 하나는, 애초에 잘못 처리하고 있다는 것이다. 이러한 식으로 너비*높이*색을 모두 조합한 경우를 처리한는 것은 그냥 나쁜 설계라고밖에 말할 수 없다. 대부분의 경우, 이보다 훨씬 잘 할 수 있는 여지가 많다.

### 성능 분석

거대한 데이터를 처리하는 알고리즘의 핵심은 일종의 분할 정복 전략을 실행하는 방법을 찾는 데 있다. 이는 함수형 설계뿐 아니라 명령형 설계에서도 마찬가지다. 앞에서 본 처리를 더 빠르게 만드는 방법이 세 가지 있다.

* 병렬성을 사용해 대부분의 계산을 동시에 처맇라 수 있다. 4코어 프로세서의 경우, 실행 시간이 1/4로 줄어들 수 있다. 그렇다면 맨해튼 거리의 경우 8분이 걸린다는 뜻이다.

* 중간 결과를 캐시에 넣어 중복 계산을 줄일 수 있다. 성능 향상은 얼마나 색이 많이 있느냐와 얼마나 많은 픽셀의 색이 유일하냐에 따라 달라진다. 

* 알고리즘을 급진적으로 바꿀 수도 있다.

마지막 두 방법응 조합하여 원래 색과 대상 색 사이의 모든 비교를 계산할 것이다. 많은 경우가 그렇듯, 이 경우에도 쉽게 전체 매핑을 열거할 수 있기 대문에 한 픽셀씩 계산할 때 하게 되는 불필요한 중복 계산을 피할 수 있다. 또한 일현의 비교를 수행하는 알고리즘을 매핑 객체에서의 단순 검색으로 바꿀 것이다.

이러한 식으로 원래의 색에서 대상 색으로의 모든 변환을 미리 계산해두는 아이디어를 생각할 때에는 임의의 이미지에 대한 전테 통계가 필요하다. 

In [12]:
from collections import defaultdict, Counter
from PIL import Image
 
img = Image.open('IMG_2705.jpg')

palette = defaultdict(list)
for xy_p in pixel_iter(img):
    xy, p = xy_p
    palette[p].append(xy)
w, h = img.size

In [13]:
print("Total pixels", w*h)
print("Total colors", len(palette))


Total pixels 3367680
Total colors 172339


주어진 색의 모든 픽셀을 모아 색으로 구분한 리스트에 넣었다. 이로부터 다음과 같은 사실을 알 수 있다.

* 전체 픽셀의 개수는 3367680개다. 

* 전체 색 개수는 172339개다. 실제 색과 133개의 색 사이의 유클리드 거리를 계산하려 시도한다해도 얼마 안걸린다.

* 3비트 마스크인 0b11100000을 사용하는 경우, 가능한 512가지 색 중에서 214색만 사용된다는 것을 알 수 있다.

* 4비트 마스크인 0b11110000을 사용하는 경우, 가능한 4096가지 색 중에서 1150색만 사용된다는 것을 알 수 있다.

* 5비트 마스크인 0b11111000을 사용하는 경우, 가능한 32768가지 색 중에서 5845색만 사용된다는 것을 알 수 있다.

* 6비트 마스크인 0b11111100을 사용하는 경우, 가능한 262144가지 색 중에서 27726색만 사용된다는 것을 알 수 있다.

이러한 통계는 수십억 번의 계산 없이 데이터 구조를 어떻게 재배열하고, 매치하는 색을 빠르게 계산하고, 다시 이미지를 재구축할 수 있는지를 생각해낼 때 몇가지 착안점을 제공한다.

다음과 같은 명령을 사용하면 마스크 값을 RGB 바이트에 적용할 수 있다.

In [15]:
masked_color = tuple(map(lambda x: x&0b11100000, c))

이렇게 하면 적, 녹, 청색 값 중 MSB 3비트만을 얻을 수 있다. 원래의 색 대신 이 값을 사용해 Counter 객체를 만든다면 214개의 서로 다른 값을 볼 수 있을 것이다.

### 문제를 다시 배열하기

product() 함수를 사용해 모든 픽셀과 모든 색을 비교한다는 단순한 생각은 좋지 않다. 원래의 색을 대상 색으로 매핑하는 방법을 선택한다면, 오직 200,000개의 값을 단순한 맵에 넣으면 된다.

다음과 같은 접근 방식을 취할 것이다.

* 원래의 색으로부터 대상 색으로의 매핑을 계산한다. 여기서는 3비트 색의 값을 출력으로 사용할 것이다. 각각의 R, G, B 값은 range(0, 256, 32)를 사용해 생성할 수 있다. 다음 식을 사용해 모든 대상 색을 열거할 것이다.

In [16]:
product(range(0, 256, 32), range(0, 256, 32), range(0, 256, 32))

* 그 후 원본 펠레트로부터 가장 가까운 색까지의 유클리드 거리를 계산할 수 있다. 단지 68,096번만 계산하면 된다. 이 계산에 0.14초가 걸린다. 거리는 오직 한 번만 계산하면 되고, 그 후 200,000개의 매핑을 게산한다.

* 이미지 전체를 한 번 훑어 나가면서 갱신한 색깔 표를 사용해 새 이미지를 만든다. 경우에 따라 정수 값의 내림 성질을 이용할 수도 있다. 

이렇게 하면 십억 개 수준의 유클리드 거리 계산을 천만 개 수준의 딕셔너리 검색으로 바꿀 수 있다. 실행헤 걸리는 시간은 30분에서 30초 정도로 바뀐다. 

모든 픽섹의 색을 매핑하는 대신, 입력 값으로부터 출력 값으로의 정적 맵을 만들 것이다. 새 이미지는 원래의 색으로부터 새로운 색으로의 매핑을 단순 검색하여 구축할 수 있다. 

200,000색의 모든 팔레트를 만들고 나면, 더 빠른 맨해튼 거리를 사용해 크레욜라 색과 같은 출력 색에서 가장 가까운 색을 찾을 수 있다. 이 과정에서 앞에서 보여줬던 색 매치 알고리즘을 전체 이미지를 계산하지 않고 팔레트에 있는 색의 매핑을 계산하도록 바꿀 수 이싿. pixel_iter() 대신 pixel.keys()를 사용하게 만드는 것이 핵심이다.

또 다른 최적화로 색 정보릐 일부를 잘라낼 수 있다. 이를 적용하면 훨씬 더 빠른 알고리즘을 만들 수 있다.

### 두 가지 변환 조합하기

여러 변환을 조합하는 경우, 원본에서 중간 결과를 거쳐 최종 결과로 가는 좀 더 복잡한 매핑을 만들 수 있다. 이를 보여주기 위해 매핑을 적용할 뿐만 아니라 색 정보의 일부를 잘라 크기를 줄일 수도 있다.

문제에 따라서 잘라내는 게 어려울 수도 있지만 종종 매우 쉽게 데이터의 일부를 잘라낼 수 있기도 하다. 예를 드렁 미국 우편번호를 아홉 자에서 다섯 자로 줄이는 경우가 자주 있다. 더 나아가 광범휘한 지역을 표시하는 세 자리 코드로 잘라낼 수도 있다.

색의 경우, 앞에서 보여준 비트 마스크를 사용하여 세 8비트 값을 세 3비트 값으로 줄일 수 있다.

다음은 주어신 색 집합과의 거리를 계산하면서 원래의 색을 잘라내어 색의 맵을 구축하는 방법을 보여준다.

In [17]:
bit3 = range(0, 256, 0b100000)

In [18]:
best = (min((euclidean(rgb, c), rgb, c) for c in colors) for rgb in product(bit3, bit3, bit3))

In [23]:
color_map = dict((b[1], b[2].rgb) for b in best)

3비트의 모든 경우에 대해 루프를 수행할 bit3이라는 range 객체를 만들었다.

range 객체는 일반적인 반복자와는 다르다. 이 객체를 여러 번 사용해도 문제가 없다. 따라서 product(bit3, bit3, bit3)는 우리가 출력 색에 사용할 모든 512가지 조합을 만들어 낼 수 있다.

각각의 3비트 RGB 색에 대해 3-튜플을 만든다. 0번 원소는 모든 크레욜라 색에 대한 거리, 1번 원소는 RGB 색, 2번 원소는 크레욜라 색의 Color 객체다. 이 3-튜플 컬렉션에 대한 최솟값을 구한다. 따라서 3비트 RGB 색과 가장 가까운 크레욜라 Color 객체를 얻는다.

3비트 RGB 색을 가장 가까운 크레욜라 색으로 매핑하는 딕셔너리를 만들었다. 이 매핑을 사용하려면 원본의 색을 3비트 색으로 바꿔야 한다. 이렇게 잘라내는 작업을 미리 계산한 매핑과 조합하는 것은 매핑 기법을 조합할 때 어떤 방식을 따라야 하는지를 보여주는 예다.

다음은 이미지 픽셀을 갱신하는 코드다.

In [31]:
clone = img.copy()
for xy, p in pixel_iter(img):
    r, g, b = p
    repl = color_map[(0b11100000&r, 0b11100000&g, 0b11100000&b)]
    clone.putpixel(xy, repl)
clone.show()

여기서 문제는 Product() 함수를 비효율적인 알고리즘에 사용했다는 점에 있다.

### 값의 컬렉션 순열 구하기

itertools.permutations() 함수를 사용하는 것은 매우 작은 문제의 해법을 탑색해봃 경우, 손십궤 사용할 수 있는 도구일 뿐이다.

이러한 조합적 최적화 문제의 가장 유명한 예는 할당 문제다. n명의 직원가 m개의 작업이 있는데, 각 직원이 어떤 작업을 수행하는 비용이 모두 같지 않다고 가정햅좌. 몇몇 직원이 아직 일부 작업의 세부 사항을 숙지하지 못한 반면, 나머지 직원들은 매우 뛰어난 경우를 상상하면 된다. 

문제 규모가 커지면 이러한 방식은 쉽게 성능이 나빠진다. 6명의 직원가 여섯 가지 작업을 조합한 36가지 값을 가지는 비용 행렬이 있는 경우, 이 문제를 다음과 같이 코딩할 수 있다.

In [37]:
from itertools import permutations
perms = permutations(range(6))
alt = [(sum(cost[x][y] for y, x in enumerate(perm)), perm)for perm in perms]

m =min(alt)[0]
print([ans for s, ans in alt if s == n])

규모가 큰 예제의 경우에는 근사칠르 구하는 알고리즘을 훨씬 적합하다. 

### 모든 조합 구하기

조합을 살펴볼 때는 원소의 순서는 중요하지 않다. 

다음은 데이터 집합에서 데이터가 있는 열을 선택하는 함수다.

In [38]:
def column(source, x):
    for row in source:
        yield row[x]